Submitting various things for end of grant.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [13]:
# live server & control file
#server = ENCODED('www.encodeproject.org')
engine='odf'
spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/10x_mouse_limb_20181219.ods')

# test server & datafile
server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/10x_mouse_limb_20181219-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [6]:
award = 'UM1HG009443'

# Submit Documents

Example Document submission

In [7]:
#atac_uuid = '0fc44318-b802-474e-8199-f3b6d708eb6f'
#atac = Document(os.path.expanduser('~/proj/encode3-curation/Wold_Lab_ATAC_Seq_protocol_December_2016.pdf'),
#                'general protocol',
#                'ATAC-Seq experiment protocol for Wold lab',
#                )
#body = atac.create_if_needed(server, atac_uuid)
#print(body['@id'])

# Submit Annotations

In [8]:
#sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#annotations = sheet.parse('Annotations', header=0)
#created = server.post_sheet('/annotations/', annotations, verbose=True, dry_run=True)
#print(len(created))

In [9]:
#if created:
#    annotations.to_excel('/tmp/annotations.xlsx', index=False)

# Register Biosamples

In [18]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [20]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [22]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/10x_mouse_limb_20181219.ods
0


In [23]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [28]:
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [29]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [31]:
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
created = server.post_sheet('/replicates/', 
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [32]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [51]:
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
submitted_name_accessions = {}
submitted_name_uuid = {}
for i, row in experiments.iterrows():
    experiment_id = '/experiments/{}/'.format(row['accession'])
    experiment = server.get_json(experiment_id)
    for f in experiment['files']:
        submitted_name_accessions[f['submitted_file_name']] = f['accession']
        submitted_name_uuid[f['submitted_file_name']] = f['uuid']


In [55]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
accessions = []
uuid = []
for i, row in files.iterrows():
    if not pandas.isnull(row['accession']):
        accessions.append(row['accession'])
        uuid.append(None)
    else:
        accessions.append(submitted_name_accessions[row['submitted_file_name']])
        uuid.append(submitted_name_uuid[row['submitted_file_name']])

In [57]:
files['accession'] = accessions
files['uuid'] = uuid
files.to_excel('/dev/shm/files.xlsx', index=False)
files

,uuid,accession,dataset,replicate,submitted_file_name,aliases:array,paired_with,file_format,output_type,run_type,paired_end,platform,lab,award
0,None,ENCFF947WRU,ENCSR787QXE,97f11d51-ce9f-4c46-90eb-7b5653a98490,wold10x_1_e13_5_mouse_limb_S1-4_L008_R1_001.fa...,barbara-wold:wold10x_1_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443
1,None,ENCFF093KXG,ENCSR787QXE,97f11d51-ce9f-4c46-90eb-7b5653a98490,wold10x_1_e13_5_mouse_limb_S1-4_L008_R2_001.fa...,barbara-wold:wold10x_1_r2_001_fastq,barbara-wold:wold10x_1_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443
2,c02de4ba-8856-49f9-9bfc-86a434d4c150,ENCFF635ZQE,ENCSR642XVO,38becebc-ee76-4d70-b753-38272b5a74c0,SE731_new/outs/fastq_path/Wold10x-3_S1_L001_R1...,barbara-wold:wold10x_3_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443
3,54fc54e2-1821-45ba-b367-2cdf5607e24c,ENCFF301CXO,ENCSR642XVO,38becebc-ee76-4d70-b753-38272b5a74c0,SE731_new/outs/fastq_path/Wold10x-3_S1_L001_R2...,barbara-wold:wold10x_3_r2_001_fastq,barbara-wold:wold10x_3_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443
4,c2b7a38f-165e-461e-b8c5-f4d502231bd8,ENCFF875QKS,ENCSR745DNK,ec3f94b3-c62b-481f-9ba0-27e30772429b,SE732_new/outs/fastq_path/Wold10x-4_S1_L002_R1...,barbara-wold:wold10x_4_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443
5,c3523337-0037-4ef6-80a0-1de1a3f0a0ce,ENCFF608GWY,ENCSR745DNK,ec3f94b3-c62b-481f-9ba0-27e30772429b,SE732_new/outs/fastq_path/Wold10x-4_S1_L002_R2...,barbara-wold:wold10x_4_r2_001_fastq,barbara-wold:wold10x_4_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443
6,103f8b2c-0116-46c1-baef-346979c74415,ENCFF190PWV,ENCSR476JWN,d4f7b5d7-e2a4-49ba-894e-7ab12209a2c2,SE733/outs/fastq_path/Wold10x-5_S1_L003_R1_001...,barbara-wold:wold10x_5_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443
7,edaa965c-5684-4914-85d4-5be2436838fb,ENCFF065KAC,ENCSR476JWN,d4f7b5d7-e2a4-49ba-894e-7ab12209a2c2,SE733/outs/fastq_path/Wold10x-5_S1_L003_R2_001...,barbara-wold:wold10x_5_r2_001_fastq,barbara-wold:wold10x_5_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443
8,fdfdc9cf-e461-421d-84a9-bcd6935ede02,ENCFF818XJI,ENCSR337QTQ,70039494-70a8-4ed7-b543-600714fb5d76,SE734/outs/fastq_path/Wold10x-6_S1_L004_R1_001...,barbara-wold:wold10x_6_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443
9,f6b22d9e-a0d6-414b-8e23-ccff7a715f4e,ENCFF167JPD,ENCSR337QTQ,70039494-70a8-4ed7-b543-600714fb5d76,SE734/outs/fastq_path/Wold10x-6_S1_L004_R2_001...,barbara-wold:wold10x_6_r2_001_fastq,barbara-wold:wold10x_6_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443


In [66]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
read_length = []
for i, row in files.iterrows():
    file_id = '/files/{}/'.format(row['accession'])
    obj = server.get_json(file_id)
    read_length.append(obj['read_length'])
files['read_length'] = read_length
files.to_excel('/dev/shm/files.xlsx', index=False)
files

,uuid,accession,dataset,replicate,md5sum,submitted_file_name,aliases:array,paired_with,file_format,output_type,run_type,paired_end,platform,lab,award,read_length
0,7d99f8fb-abf5-41dc-87fc-56001a10f6bf,ENCFF947WRU,ENCSR787QXE,97f11d51-ce9f-4c46-90eb-7b5653a98490,0dc6fea6229425ebbc579f3f7e51208f,wold10x_1_e13_5_mouse_limb_S1-4_L008_R1_001.fa...,barbara-wold:wold10x_1_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443,151
1,aff7fd00-59f8-4815-bac9-394ca8473f1d,ENCFF093KXG,ENCSR787QXE,97f11d51-ce9f-4c46-90eb-7b5653a98490,1190063ba22554079b5af599daca1b12,wold10x_1_e13_5_mouse_limb_S1-4_L008_R2_001.fa...,barbara-wold:wold10x_1_r2_001_fastq,barbara-wold:wold10x_1_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443,151
2,c02de4ba-8856-49f9-9bfc-86a434d4c150,ENCFF635ZQE,ENCSR642XVO,38becebc-ee76-4d70-b753-38272b5a74c0,b6ae15afdacf360d5402892f8b546d14,SE731_new/outs/fastq_path/Wold10x-3_S1_L001_R1...,barbara-wold:wold10x_3_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443,151
3,54fc54e2-1821-45ba-b367-2cdf5607e24c,ENCFF301CXO,ENCSR642XVO,38becebc-ee76-4d70-b753-38272b5a74c0,030db377778c53dcf96f63f6a6e57f53,SE731_new/outs/fastq_path/Wold10x-3_S1_L001_R2...,barbara-wold:wold10x_3_r2_001_fastq,barbara-wold:wold10x_3_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443,151
4,c2b7a38f-165e-461e-b8c5-f4d502231bd8,ENCFF875QKS,ENCSR745DNK,ec3f94b3-c62b-481f-9ba0-27e30772429b,ac527fc5911fc31c1aba76443778dd8a,SE732_new/outs/fastq_path/Wold10x-4_S1_L002_R1...,barbara-wold:wold10x_4_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443,151
5,c3523337-0037-4ef6-80a0-1de1a3f0a0ce,ENCFF608GWY,ENCSR745DNK,ec3f94b3-c62b-481f-9ba0-27e30772429b,10bd6c9d8984c38974a65fec219640c7,SE732_new/outs/fastq_path/Wold10x-4_S1_L002_R2...,barbara-wold:wold10x_4_r2_001_fastq,barbara-wold:wold10x_4_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443,151
6,103f8b2c-0116-46c1-baef-346979c74415,ENCFF190PWV,ENCSR476JWN,d4f7b5d7-e2a4-49ba-894e-7ab12209a2c2,17b129b987ee30bd04bcd1cae380b1db,SE733/outs/fastq_path/Wold10x-5_S1_L003_R1_001...,barbara-wold:wold10x_5_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443,151
7,edaa965c-5684-4914-85d4-5be2436838fb,ENCFF065KAC,ENCSR476JWN,d4f7b5d7-e2a4-49ba-894e-7ab12209a2c2,7613646e2ab71156b1860ceae9ed0550,SE733/outs/fastq_path/Wold10x-5_S1_L003_R2_001...,barbara-wold:wold10x_5_r2_001_fastq,barbara-wold:wold10x_5_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443,151
8,fdfdc9cf-e461-421d-84a9-bcd6935ede02,ENCFF818XJI,ENCSR337QTQ,70039494-70a8-4ed7-b543-600714fb5d76,b26b1e55f5dbd20cfa7aad86b3766600,SE734/outs/fastq_path/Wold10x-6_S1_L004_R1_001...,barbara-wold:wold10x_6_r1_001_fastq,NaN,fastq,reads,paired-ended,1,encode:HiSeq4000,barbara-wold,UM1HG009443,149
9,f6b22d9e-a0d6-414b-8e23-ccff7a715f4e,ENCFF167JPD,ENCSR337QTQ,70039494-70a8-4ed7-b543-600714fb5d76,d81378134bc244f8c5228e24550ffc8a,SE734/outs/fastq_path/Wold10x-6_S1_L004_R2_001...,barbara-wold:wold10x_6_r2_001_fastq,barbara-wold:wold10x_6_r1_001_fastq,fastq,reads,paired-ended,2,encode:HiSeq4000,barbara-wold,UM1HG009443,151


In [64]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
for i, row in files.iterrows():
    obj = server.get_json(row['aliases:array'])
    print(i, obj['uuid'], obj['accession'])

Error http status: 404 for https://test.encodedcc.org/barbara-wold:wold10x_1_r1_001_fastq


HTTPError: 404 Client Error: Not Found for url: https://test.encodedcc.org/barbara-wold:wold10x_1_r1_001_fastq?limit=all

In [68]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

0
